In [ ]:
%pylab inline

In [ ]:
import numpy as np
import scipy.linalg as la

## Forward PFB

The goal of the PFB is to spit a stream of electric eld data up into frequency channels, where the response inside a channel is at, and the response outside
the channel dies as quickly as possible. If we had a very long electric field timestream, the ideal thing to do would be to Fourier transform the electric field, then cut out a boxcar in Fourier space. The PFB tries to do this as well as possible and as efficiently as possible given the real-world constraints of digital signal processing.

A classic DFT will end up convoloving a box car function in frequency domain thus the first step in a pfb is to deconvolve with the box car in frequency domain which happens to be a sinc in time domain. Thus we create a de-convolving function which is a sinc funnction multiplied by a window (usually Hamming or Hanning) for out of band rejection.

The second step is to effectivly average a couple bins together to get a more flat response inside a given frequency bin. This is done by averaging $n_{tap}$ time domain points together spaced $2n_{chan}$ points appart.

Put together this means we take $2n_tap n_chan$ points of time stream, multiply it by the window function and then sum points $2n_{chan}$ appart to get a psudo data stream $2n_{chan}$ long which will be DFTed to get $n_{chan}$ frequency bins.

If one desires to PFB a chunk of data longer than $2n_{tap} n_{chan}$ then the same procedure as above is repeated but the data used is shifted over by $2 n_{chan}$ such that there is always overlap between the chunks being PFBed

### The Math 

Now lets break up the above ramble into linear operations. First thing we need is the window function $W$ which multiplies a chunk of data $d$. Taking $d$ to be an $N = 2 n_{chan} n_{tap}$ dimensional vector we see that $W$ must be a $N \times N$ diagonal matrix with eignevalues given by the window coeficients.

Next the windowed time stream $d_w = Wd$ needs to be summed as explained above by a matrix $S$. To do this we will need a $ 2 n_{chan} \times 2 n_{chan} n_{tap}$ matrix which is Toeplitz with a diagonal full of ones and the $2tn_{chan}, \ t=1,\dots, n_{tap}$ super diagonal are also full of ones.

Finally the windowed and summed time stream $d_{sw} = SWd$ needs to be DFTed by the traditional DFT represented by $F$. Thus the PFB is simply a set of linear operations as follows:
$$ PFB(d) = FSWd$$

In [ ]:
## Lets make the matricies one at a time
## starting with W

##firs define the sinc window function:
def sinc_window(ntap, lblock):
    x = np.arange(-ntap/2, ntap/2, 1/lblock)
    return np.sinc(x) # this is actually sinc(pi*x)
##now wrap the hanning window
def sinc_hanning(ntap, lblock):
    return np.hanning(ntap * lblock)

##put the 2 together to get:
def get_window(ntap, lblock, window = sinc_hanning):
    return sinc_window(ntap, lblock) * window(ntap, lblock)

In [ ]:
##now to make it a matrix:
from scipy.sparse import diags
def window_mat(ntap, lblock):
    N = ntap * lblock
    mat = diags(get_window(ntap, lblock)).toarray()
    return mat

In [ ]:
window_mat(2,3)

In [ ]:
##now onto S
def sum_mat(ntap, lblock):
    col = np.zeros(lblock)
    col[0] = 1
    row = np.zeros(ntap*lblock)
    row[::lblock] = 1
    return la.toeplitz(col, row)

In [ ]:
sum_mat(2,6)

In [ ]:
## finally we want F
def dft_mat(lblock):
    return la.dft(lblock)

In [ ]:
dft_mat(4)

In [ ]:
## and we have PFB = FSW
def get_PFB_full_mat(ntap,lblock):
    return np.dot(dft_mat(lblock),np.dot(sum_mat(ntap, lblock), window_mat(ntap, lblock)))


In [ ]:
##so now to do the transform we do:
def PFB_full_mat(data, nchan, ntap = 4):
    lblock = 2*nchan
    nblock = data.size // lblock -(ntap -1)
    
    #init the results
    spec = np.zeros((nblock, nchan), dtype=np.complex128)

    #init mat
    mat = get_PFB_full_mat(ntap, lblock)

    #go through 1 chunk at a time:
    for chunk in range(nblock):
        data_chunk = data[(chunk *lblock):((chunk + ntap) * lblock)]

        spec[chunk,:] = np.dot(mat, data_chunk)[:nchan]
    return spec

In [ ]:
ntime = 2**11
ta = np.linspace(0.0, ntime / 2048, ntime, endpoint=False)

ts = np.sin(2*np.pi * ta * 122.0) + np.sin(2*np.pi * ta * 378.1 + 1.0)

In [ ]:
spec_pfb = PFB_full_mat(ts, 17, ntap=4)

In [ ]:
imshow(np.abs(spec_pfb), aspect='auto', interpolation='nearest')
colorbar()

In [ ]:
plot(np.abs(spec_pfb[10]))

## Inverse PFB

Here we are caught in a bit of a pickle. The first thing to try to invert the PFB would be to invert the PFB matrix. But it is not diagonal and thus the system is unsolvable (The PFB linear map is $\mathcal{R}^{2n_{chan} n_{tap}}\rightarrow \mathcal{C}^{n_{chan}}$ hence clearly can not be simply inverted). 

But all is not lost. First of all, $F^{-1}$ is well defined so we can start by undoing that. Denoting the result of the pfb by $P$ we have:

$$P = FSWd \\ F^{-1}P = SWd$$

This has yet to fix the out of square problem we had before but all is not lost. Lets imagine we performed the PFB of a very large amount of data, say $N$ where $N >> 2*n_{tap}*n_{chan}$. In this case we performed the linear PFB map on many chunks of data. And recall that while $n_{tap}$ points get summed together (thus creating this problem in the first place) the PFB map is performed in sequence on blocks of the time stream such that each point is used $n_{tap}$ times. Thus as $N$ gets very large the number of output points tends to match the number of input points. Thus in the middle of the timestream where points are used $n_{tap}$ times we should have no problem inverting the PFB and we will talk about edge effects later.

So lets consider the matrix $SW$. This matrix is stack diagonal and to understand what it does lets recall what $S$ and $W$ do individually. $W$ simply is identity and weighs the input timestream (in linear terms it stretches space to colaps some dimesions and extend others) and $S$ sums every $2n_{chan}$ points (or in linear terms it collaps every $2n_{chan}$ dimension together). Hence $SW$ will look exactly like $S$, that is, it will be sparce with the $2tn_{chan}, \ t=0, 1,\dots, n_{tap}$ super diagonal being full of the eignvalues of $W$.

Hence after doing the invers DFT we are left with $2n_{chan}$ independent problems. So lets consider a single one of these problems: the $2tn_{chan}, \ t=1,2,3, \dots$ timestream points. Consider some point of this psudo timestream and how $SW$ affected it. The first time the PFB was run (on a block of data containing it), it added that datapoint weighed by the $2 * n_{chan} * (n_{tap} -1)$, the second time it added that data point weighed by $ 2 * n_{chan} * (n_{tap} -1)$ and so on. So the $t$ time it was run it added $ 2 * n_{chan} * (n_{tap} -t)$ to the resultent psudo timestream. And this is true for all the points! Only differing when they entered and exited the block being evaluated. Thus we see that expressing the $2tn_{chan}, \ t=1,2,3, \dots$ timestream points as $d_p$ and the effective $SW$ acting on them becomes a band diagonal toeplitz matrix called $SW_p$. 

We are kinda done since we can "almost" invert this matrix minus for the ends where points were not used $n_{tap}$ times. A propsed solution (which is not the only one) is to create circulant boundary conditions. Thus taking the first few blocks and repeating them at the end to create a square matrix which is already by construction circulant. This gives us a couple great advantages like being able to invert this sparce matrix by convolving it with the psudo inverse denoted as $d_p$. Computationally this gives us the ability to perform it in $\mathcal{O}(n\ln(n))$ time with a really good factor in front of it by using the FFT implementation of the DFT!

To get the full inverse we solve this problem $2n_{chan}$ times thus solving all the independent (unmixed) points

In [ ]:
def inverse_pfb_fft(data, ntap, window_func = sinc_hanning):
    ##undo the DFT as it was applied so along the horizontals
    ##thus vertical columns of this matrix represent mixed points as discribed above
    psudo_ts = np.fft.irfft(data, axis=1)
    
    ##get the window function vals and shape it 
    window_mat = get_window(ntap, psudo_ts.shape[1], window = window_func).reshape(ntap, -1)
    
    #init the SW_p matrix
    SW_p = np.zeros(psudo_ts.shape, dtype=psudo_ts.dtype)
    SW_P[:ntap,:] = window_mat

    ##switch into freq domain
    SW_p_Ft = np.fft.rfft(SW_p, axis = 0)
    window_mat_Ft = np.fft.rfft(window_mat, axis = 0)

    ##deconvolve:
    ts_Ft = SW_p_Ft / np.conj(window_mat_Ft)

    ##return to time domain
    reconstructed_ts = np.fft.irfft(ts_Ft, axis=0)

    return reconstructed_ts

lets leave a little note as to why this is a strange matrix and not the simple matrix we were promised which was sparce. 